In [1]:
import requests
import re
from bs4 import BeautifulSoup as BS
import pandas as pd
import datetime
import csv

In [ ]:

evo = "https://www.evo.com/shop/snowboard/snowboards/rpp_400"
headers = {'User-agent': 'Mozilla/5.0'}
response = requests.get(evo, headers = headers)

In [ ]:
response

<Response [200]>

In [ ]:
soup = BS(response.content,'html.parser')


In [ ]:
items = soup.find_all('div',{'class':'product-thumb-details'})


In [ ]:
def company_cleaner (title_str):
  names = []
  for snowboard in items:
    names.append((snowboard.find('span',{'class':'product-thumb-title'}).text).split(' ')[0])

  return names

def board_cleaner(title_str):
  names = []
  for snowboard in items:
    names.append((snowboard.find('span',{'class':'product-thumb-title'}).text))
  return names


In [ ]:
brands = company_cleaner(items)
boards = board_cleaner(items)

In [ ]:
def price_finder(soups):
  price_pattern = r'([\d+\,]+\d+\.\d{1,2})'
  price_list = []
  for slop in soups:
    price_list.append((re.findall(price_pattern, str(slop))))
  return price_list

In [ ]:
stringprices = price_finder(items)

In [ ]:
#need to regex remove ',' from $1000+ items in price_finder then convert str to float
def price_cleaner(prices):
  finalprice=[]
  for pricelist in prices:
    rinsedprice =[]
    finalprice.append(rinsedprice)
    for price in pricelist:
      rinsedprice.append(float(price.replace(',','')))
  return finalprice  

In [ ]:
prices = price_cleaner(stringprices)
prices[0]

[499.0, 499.0, 349.3]

In [ ]:
collected = [str(datetime.date.today())]*len(boards)

In [ ]:
complete = list(zip(collected, boards, brands))
complete[0]

('2023-03-29', 'Season Kin Snowboard 2023', 'Season')

In [ ]:
priceframe = pd.DataFrame(prices, columns = 'main dropprice sale non'.split())
priceframe.head()

,main,dropprice,sale,non
0,499.00,499.00,349.30,NaN
1,329.95,329.95,263.96,NaN
2,899.95,899.95,764.96,NaN
3,599.95,599.95,479.96,NaN
4,1199.99,1199.99,959.99,NaN


In [ ]:
del priceframe['dropprice']
del priceframe['non']

In [ ]:
boardframe = pd.DataFrame(complete, columns = 'Collected Deck Brand'.split())

In [ ]:
df = pd.concat([boardframe,priceframe], axis = 1, sort = 'False')

<ipython-input-17-98d83e2bdb64>:1: FutureWarning: Passing non boolean values for sort is deprecated and will error in a future version!
  df = pd.concat([boardframe,priceframe], axis = 1, sort = 'False')


In [ ]:
df['Collected'] = pd.to_datetime(df.Collected)

In [ ]:
df.head()

,Collected,Deck,Brand,main,sale
0,2023-03-29,Season Kin Snowboard 2023,Season,499.00,349.30
1,2023-03-29,Nitro Prime Raw Snowboard 2023,Nitro,329.95,263.96
2,2023-03-29,Jones Hovercraft Splitboard 2023,Jones,899.95,764.96
3,2023-03-29,Arbor Coda Rocker Snowboard 2023,Arbor,599.95,479.96
4,2023-03-29,Lib Tech T.Rice Apex Orca Snowboard 2023,Lib,1199.99,959.99


In [ ]:
df.dtypes

Collected    datetime64[ns]
Deck                 object
Brand                object
main                float64
sale                float64
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Collected  400 non-null    datetime64[ns]
 1   Deck       400 non-null    object        
 2   Brand      400 non-null    object        
 3   main       400 non-null    float64       
 4   sale       333 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 15.8+ KB


In [ ]:
df.to_csv('my_data.csv', mode='a', header=False, index=False)